In [104]:
import requests
import urllib.parse
import json
import pandas as pd
from pandas import json_normalize 
import csv
from datetime import datetime
import zipcodes

In [2]:
#Vars and Stuff--------------------------------------------------------------------------------------------------------------------------------------
AUTH_URL = "https://auth.bullhornstaffing.com/oauth"
REST_URL = "https://rest.bullhornstaffing.com/rest-services"

auth = dict(
    client = 'NA',
    secret = 'NA',
    username = "NA",
    password = "NA")

#BHAuth Stuff----------------------------------------------------------------------------------------------------------------------------------------

def get_auth_code():
    params = dict(
            client_id= auth['client'],
            response_type= "code",
            username= auth['username'],
            password= auth['password'],
            action= "Login")

    url = AUTH_URL + "/authorize"
    req = requests.get(url, params=params)
    response_url = req.url
    response_query = requests.utils.urlparse(response_url).query
    #Parses return URL to get us just the code.
    query_dict = urllib.parse.parse_qs(response_query)
    if query_dict.get("code"):
        auth_code = query_dict["code"][0]
    else:
        auth_code = ""
    return auth_code


def get_access_token(auth_code):
    params = dict(
            client_id=auth['client'],
            client_secret=auth['secret'],
            code = auth_code,
            grant_type="authorization_code")
    
    url = AUTH_URL + "/token"

    req = requests.post(url, params=params)
    access_token_url = req.url
    try:
        response_dict = req.json()
    except Exception as err:
        print(err)
        response_dict = {}
    return(response_dict)
#BHRest Stuff----------------------------------------------------------------------------------------------------------------------------------------

version = "*"

import pprint
def rest_login(access_token):
    url = REST_URL +"/login"
    login_params = dict(
            version = version,
            access_token = access_token['access_token'])
    req = requests.get(url, params=login_params)
    try:
        response_dict = req.json()
    except Exception as err:
        print(err)
        rest_login = {}
    rest_login = response_dict
    return rest_login

#Main------------------------------------------------------------------------------------------------------------------------------------------------
def main():
    auth_code = get_auth_code()
    access_tokenDict = get_access_token(auth_code)
    BhRestTokenDict = rest_login(access_tokenDict)
    return BhRestTokenDict

In [3]:
def getCandidates():

    auth_get = main()
    auth_url = auth_get['restUrl']
    auth_token = auth_get['BhRestToken']
    
    df = pd.DataFrame()

    start = 0
    count = 500

    while count != 0: 

        query = requests.get(auth_url+"search/Candidate?query=isDeleted:0&fields=firstName,lastName,id,occupation,address,companyName,dateAdded,source&start="+str(start)+"&count=500&BhRestToken="+auth_token)
        query_json = query.json()

        if 'errorMessage' in query_json:

            print("reauthenticating...")

            auth_get = main()
            auth_url = auth_get['restUrl']
            auth_token = auth_get['BhRestToken']

            query = requests.get(auth_url+"search/Candidate?query=isDeleted:0&fields=firstName,lastName,id,occupation,address,companyName&start="+str(start)+"&count=500&BhRestToken="+auth_token)
            query_json = query.json()

            count = query_json['count']
            start = query_json['start'] + count
            
            df = pd.concat([df, json_normalize(query_json['data'])]) 

        else: 
            count = query_json['count']
            start = query_json['start'] + count

            df = pd.concat([df, json_normalize(query_json['data'])])
        
        print(start)

    print("query finished.")

    return df

In [ ]:
df = getCandidates()

In [ ]:
df.to_csv(r'C:/Users/other/Documents/Capstone/Data/all_cns_8_1_23.csv', index=False, header=True)